# Объединение датафреймов. 
1. Самый простой случай - когда оба датафрейма имеют одинаковый формат. В этом случае достаточно выполнить конкатенацию с помощью метода pd.concat().
2. Конкатенация - это просто "склеивание" двух датафреймов по колонкам. 

In [21]:
import numpy as np 
import pandas as pd

In [22]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

In [23]:
one = pd.DataFrame(data_one)
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [24]:
two = pd.DataFrame(data_two)
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


In [25]:
pd.concat([one,two], axis=1)


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [26]:
pd.concat([one,two],axis=0)

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


In [27]:
# переименовать названия признаков. 
one,two

(    A   B
 0  A0  B0
 1  A1  B1
 2  A2  B2
 3  A3  B3,
     C   D
 0  C0  D0
 1  C1  D1
 2  C2  D2
 3  C3  D3)

In [28]:
two.columns = one.columns

In [29]:
# с повторами индексов
pd.concat([one,two], axis=0)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


In [30]:
# без повторов - нужно сохранить в датафрейм. 
mydf = pd.concat([one,two], axis=0)
mydf.index = range(len(mydf)) # указываем новые значения для индекса. 
mydf

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
4,C0,D0
5,C1,D1
6,C2,D2
7,C3,D3


---
1. Зачастую датафреймы имеют разный формат, и поэтому их нельзя объединять с помощью конкатенации. 
2. В этом случае применяется слияние - merge. 
3. Это действие аналогично операции JOIN для соед. таблиц в SQL.

---
* Метод .merge() принимает на вход параметр how
* Есть 3 способа: inner, outer, left and right.
* inner - соответсвующие строки должны присутствовать в 2 таблицах. Берет пересечение колонки. 

In [31]:
mydf

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
4,C0,D0
5,C1,D1
6,C2,D2
7,C3,D3


In [33]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})
registrations, logins

(   reg_id    name
 0       1  Andrew
 1       2    Bobo
 2       3  Claire
 3       4   David,
    log_id     name
 0       1   Xavier
 1       2   Andrew
 2       3  Yolanda
 3       4     Bobo)

In [36]:
pd.merge(registrations, logins, how='inner', on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


---
* left & right merge. Все значения "left | right" таблицы + пересечение 2 таблиц. 

In [38]:
pd.merge(registrations, logins, how='left', on='name')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


In [39]:
pd.merge(registrations, logins, how='right', on='name')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


---
* outer merge() - объединение значении 2 таблиц. 

In [40]:
pd.merge(registrations, logins, how='outer', on='name')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


---
* меняем местами

In [42]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [45]:
# rename columns 
registrations.columns = ['reg_id','reg_name']

In [46]:
registrations

,reg_id,reg_name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [47]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [49]:
# объединение по колонке с разными названиями
results = pd.merge(registrations, logins, how='inner', left_on='reg_name', right_on='name')

In [50]:
results.drop('name', axis=1)

,reg_id,reg_name,log_id
0,1,Andrew,2
1,2,Bobo,4


---
* Работа с дубликатами колонок

In [62]:
registrations = registrations.reindex(columns=['name','id'])
registrations

,name,id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [59]:
logins.columns = ['id', 'name']
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [64]:
pd.merge(registrations, logins, how='inner', on='name', suffixes=('a'))

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [65]:
# меняем суффиксы
pd.merge(registrations, logins, how='inner', on='name', suffixes=('_reg','_log'))

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4
